# POC Pushing a file into Google Drive using the Google API

In [ ]:
from __future__ import print_function
import pickle
import os
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.http import MediaFileUpload

# Method 1 - follow the Documentation Tutorial
Doesn't Oauth2 so it's not helpful for a automated solution

In [ ]:
os.remove('token.pickle')

In [ ]:
# If modifying these scopes, delete the file token.pickle.
# SCOPES = ['https://www.googleapis.com/auth/drive.metadata.readonly']
SCOPES = ['https://www.googleapis.com/auth/drive', 
          'https://www.googleapis.com/auth/drive.file']

"""Shows basic usage of the Drive v3 API.
Prints the names and ids of the first 10 files the user has access to.
"""
creds = None
# The file token.pickle stores the user's access and refresh tokens, and is
# created automatically when the authorization flow completes for the first
# time.
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

In [ ]:
# List the files in Google Drive
service = build('drive', 'v3', credentials=creds)

# Call the Drive v3 API
results = service.files().list(
    pageSize=10, fields="nextPageToken, files(id, name)").execute()
items = results.get('files', [])

if not items:
    print('No files found.')
else:
    print('Files:')
    for item in items:
        print(u'{0} ({1})'.format(item['name'], item['id']))

In [ ]:
# Copy this file into Google Drive
with open('the_file.txt', 'w') as f:
    f.write('hello world - v2')

# From Google Drive URL https://drive.google.com/drive/folders/fjio34nri32o4839f2FEDSfdsa_userYourOwnHere
# If rerunning, be sure to update this directory
destination_folder_id = 'fjio34nri32o4839f2FEDSfdsa_userYourOwnHere'

file_metadata = {
              'parents' : [destination_folder_id],
              'name': 'the_file.txt'}
media = MediaFileUpload('the_file.txt')
file = service.files().create(body=file_metadata, media_body=media, fields='id').execute()


In [ ]:
# Make a new directory and upload the file there
# Running this a second time will just make a new directory
file_metadata = {
    'name': 'new_folder_1',
    'parents': [destination_folder_id],
    'mimeType': 'application/vnd.google-apps.folder'
}
file = service.files().create(body=file_metadata,
                                    fields='id').execute()
subdir_file_id = file.get('id')

file_metadata = {
    'name': 'the_file.txt',
    'parents': [subdir_file_id]
}
media = MediaFileUpload('the_file.txt')
file = service.files().create(body=file_metadata,
                                    media_body=media,
                                    fields='id').execute()

# Method 2 - Use OAuth 2.0 for an automated solution

In [ ]:
# Steps
# 1. https://console.developers.google.com/iam-admin/serviceaccounts
# 2. Create Project - dora-dev
# 3. Create Service Account - dora-dev
# 4. Continue - Done
# 5. Click on Service Account - Add Key - Create New Key - JSON - Download
# 6. key_file_location below should point to this file
# 7. Search for Google Drive API and Enable it

# Directions are also here - https://help.talend.com/reader/E3i03eb7IpvsigwC58fxQg/EjqPCVhQjCFPP6pU5Bzvdw

from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

# What other scopes are there? https://developers.google.com/identity/protocols/oauth2/scopes#drive
scopes = ['https://www.googleapis.com/auth/drive'
          , 'https://www.googleapis.com/auth/drive.file']

key_file_location = '/Users/stephenpettinato/September/dora-dev-290518-2d196386bb81.json'

creds = ServiceAccountCredentials.from_json_keyfile_name(key_file_location, scopes=scopes)

service2 = build('drive', 'v3', credentials=creds)
print('connection OK')

In [ ]:
# Make a new directory and upload the file there
# Running this a second time will just make a new directory
# Make sure your service account has access to this directory
# 1. https://console.developers.google.com/iam-admin/serviceaccounts
# 2. Click on your project
# 3. Copy the email address
# 4. Go to the directory location
# 5. Enable Editor access for this service account email address

# https://drive.google.com/drive/folders/fjio34nri32o4839f2FEDSfdsa_userYourOwnHere
destination_folder_id = 'fjio34nri32o4839f2FEDSfdsa_userYourOwnHere'

file_metadata = {
    'name': 'new_folder_1',
    'parents': [destination_folder_id],
    'mimeType': 'application/vnd.google-apps.folder'
}
file = service2.files().create(body=file_metadata,
                                    fields='id').execute()
subdir_file_id = file.get('id')

file_metadata = {
    'name': 'the_file.txt',
    'parents': [subdir_file_id]
}
media = MediaFileUpload('the_file.txt')
file = service2.files().create(body=file_metadata,
                                    media_body=media,
                                    fields='id').execute()